# Informer Nuovi Positivi, Terapia Intensiva, Deceduti

In questo notebook sono stati utlizzati gli stessi dati del notebook https://colab.research.google.com/github/marco-mazzoli/progetto-tesi/blob/master/covid_multi_output.ipynb, ossia quelli regionali dell'Emilia-Romagna.
Sono in particolare stati usati i dati di ricoverati in terapia intensiva, deceduti e nuovi positivi fino al 6 Dicembre 2021.

Il modello Informer è stato utilizzato in modalità univariate con lag di 28 giorni per fare forecasting di 1, 2, 7 e 14 giorni.

L'ultimo 20% dei dati è stato escluso dalla fase di ricerca degli iper-parametri. Una volta stabiliti è stato usato l'intero dataset per fare previsione mantendo il primo 80% di train e il restante di test.

Prima della fase di train i dati vengono scalati per poi essere riportati alla scala originale una volta effettuata la previsione.


In [ ]:
!git clone https://github.com/zhouhaoyi/Informer2020.git

fatal: destination path 'Informer2020' already exists and is not an empty directory.
fatal: destination path 'ETDataset' already exists and is not an empty directory.
covid.csv  covid_holdout.csv  ETDataset  Informer2020  sample_data


In [ ]:
import sys
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
from data.data_loader import Dataset_Pred
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from data.data_loader import Dataset_Custom
from torch.utils.data import DataLoader
import pandas as pd
import os
import numpy as np
from utils.metrics import MAE
import plotly.graph_objects as go
from pandas.io.parsers import read_csv


In [ ]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        
def select_relevant_rows(frame, row, filter):
    return frame[frame[row] == filter]

def select_attributes(frame, attributes):
    return frame[attributes]

In [ ]:
use_existing_config = True
column_to_predict = 'terapia_intensiva'
columns = ['nuovi_positivi', 'terapia_intensiva', 'deceduti']
split_percent = 0.80
n_futures = [1, 2, 7, 14]
region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'
local_region_path = r'../COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'
remote_region_path = r'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

regions_frame = pd.read_csv(remote_region_path)

region_focus_data = select_relevant_rows(
    regions_frame,
    attribute_focus,
    region_focus
)

df = select_attributes(region_focus_data, [
    'data',
    'ricoverati_con_sintomi',
    'terapia_intensiva',
    'totale_ospedalizzati',
    'variazione_totale_positivi',
    'nuovi_positivi',
    'deceduti',
    'tamponi',
    'ingressi_terapia_intensiva'
])

# YYYY-MM-DD hh:mm:ss

df = pd.DataFrame(df)
df['data'] = pd.to_datetime(df['data'], format=r'%Y-%m-%dT%H:%M:%S')
df['data'] = df['data'].dt.strftime(r'%Y-%m-%d')
df['data'] = pd.to_datetime(df['data'])
df = df.fillna(0)
df.rename(columns={'data': 'date'}, inplace=True)

# revert cumulative data
df['deceduti'] = df['deceduti'].diff()
df['tamponi'] = df['tamponi'].diff()
df.dropna(inplace=True)

df = df[df['date'] < pd.to_datetime('2021-12-22')]

split = int(split_percent*len(df))

df_holdout = df[:split]

df.to_csv('./covid.csv', index=False)
df_holdout.to_csv('./covid_holdout.csv', index=False)


In [ ]:
def define_informer_configs():
    layer_encoder = [4, 3, 2]
    layer_decoder = [2]
    multi_head_number = [8, 16]
    multi_head_output = [512]
    objectives = ['mse']
    input_seq = [28, 56, 112]
    decoder_start_tok = [28, 56, 112]
    
    configs = []
    keys = ['layer_encoder', 'layer_decoder', 'multi_head_number', 'multi_head_output',
            'input_seq', 'objective', 'decoder_start_tok']

    for i in layer_encoder:
        for j in layer_decoder:
            for k in multi_head_number:
                for l in multi_head_output:
                    for m in input_seq:
                            for n in objectives:
                                    for o in decoder_start_tok:
                                        config = dict(
                                            zip(keys, (i, j, k, l, m, n, o)))
                                        configs.append(config)

    return configs

In [ ]:
def prepare_args(config, path, column_to_predict, n_future):
    args = dotdict()
    args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]
    args.data = 'custom' # data
    args.root_path = '' # root path of data file
    args.data_path = path # data file
    args.features = 'S' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
    args.target = column_to_predict # target feature in S or MS task
    args.freq = 'd' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
    args.checkpoints = './informer_checkpoints' # location of model checkpoints

    args.seq_len = config['input_seq'] # input sequence length of Informer encoder
    args.label_len = n_future #config['decoder_start_tok'] start token length of Informer decoder
    args.pred_len = n_future # prediction sequence length
    # Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

    args.enc_in = 1 # encoder input size
    args.dec_in = 1 # decoder input size
    args.c_out = 1 # output size
    args.factor = 5 # probsparse attn factor
    args.d_model = 512 # dimension of model
    args.n_heads = config['multi_head_number'] # num of heads
    args.e_layers = config['layer_encoder'] # num of encoder layers
    args.d_layers = config['layer_decoder'] # num of decoder layers
    args.d_ff = 2048 # dimension of fcn in model
    args.dropout = 0.05 # dropout
    args.attn = 'prob' # attention used in encoder, options:[prob, full]
    args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
    args.activation = 'gelu' # activation
    args.distil = True # whether to use distilling in encoder
    args.output_attention = False # whether to output attention in ecoder
    args.mix = True
    args.padding = 0

    args.batch_size = 30
    args.learning_rate = 0.0001
    args.loss = config['objective']
    args.lradj = 'type1'
    args.use_amp = False # whether to use automatic mixed precision training

    args.num_workers = 0
    args.itr = 1
    args.train_epochs = 6
    args.patience = 3
    args.des = 'exp'
    args.scale = True
    args.inverse = True

    args.use_gpu = True if torch.cuda.is_available() else False
    args.gpu = 0

    args.use_multi_gpu = False
    args.devices = '0,1,2,3'

    args.do_predict = True

    args.freq = args.freq[-1:]

    return args

In [ ]:
for column in columns:
    for n_future in n_futures:
        configs = define_informer_configs()

        results = []

        for config in configs:
            Exp = Exp_Informer
            args = prepare_args(config, 'covid_holdout.csv', column, n_future)

            setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(
                args.model, args.data, args.features, args.seq_len, args.label_len, 
                args.pred_len, args.d_model, args.n_heads, args.e_layers,
                args.d_layers, args.d_ff, args.attn, args.factor, args.embed, 
                args.distil, args.mix, args.des, args.itr)

            with HiddenPrints():      
                exp = Exp(args)
                try:
                    exp.train(setting)
                except Exception:
                    continue
                exp.test(setting)

            torch.cuda.empty_cache()

            preds = np.load('./results/'+setting+'/pred.npy')
            trues = np.load('./results/'+setting+'/true.npy')

            mae = (MAE(trues, preds))

            results.append((mae, config))

        results.sort(key=lambda tup: tup[0])

        best_config = results[0][-1]

        Exp = Exp_Informer
        args = prepare_args(best_config, 'covid.csv', column, n_future)

        setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                    args.seq_len, args.label_len, args.pred_len,
                    args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, args.itr)

        with HiddenPrints():
            exp = Exp(args)
            exp.train(setting)
            exp.test(setting)

        torch.cuda.empty_cache()

        preds = np.load('./results/'+setting+'/pred.npy')
        trues = np.load('./results/'+setting+'/true.npy')

        errors = []

        for i in range(len(trues)):
            current_error = MAE(trues[i,:,0], preds[i,:,0])
            errors.append(current_error)

        errors.sort()

        print(column + ' seq len ' + str(n_future) + ' best config: ')
        print(best_config)
        print('best error')
        print(errors[0])
        print('mae:')
        print(MAE(trues, preds))

        if n_future > 2:
            prediction_trace = go.Scatter(y=preds[0,:,0], mode='lines', name='Prediction')
            truth_trace = go.Scatter(y=trues[0,:,0], mode='lines', name='Ground Truth')
            layout = go.Layout(
                title=column, xaxis={'title': 'Date'},
                yaxis={'title': column})
            fig = go.Figure(
                data=[prediction_trace, truth_trace], layout=layout)
            fig.show()


nuovi_positivi seq len 1 best config: 
{'layer_encoder': 2, 'layer_decoder': 2, 'multi_head_number': 8, 'multi_head_output': 512, 'input_seq': 28, 'objective': 'mse', 'decoder_start_tok': 28}
best error
1.8229065
mae:
118.748505
nuovi_positivi seq len 2 best config: 
{'layer_encoder': 4, 'layer_decoder': 2, 'multi_head_number': 8, 'multi_head_output': 512, 'input_seq': 28, 'objective': 'mse', 'decoder_start_tok': 56}
best error
4.627716
mae:
90.7372
nuovi_positivi seq len 7 best config: 
{'layer_encoder': 4, 'layer_decoder': 2, 'multi_head_number': 8, 'multi_head_output': 512, 'input_seq': 112, 'objective': 'mse', 'decoder_start_tok': 28}
best error
41.94526
mae:
162.37643


nuovi_positivi seq len 14 best config: 
{'layer_encoder': 4, 'layer_decoder': 2, 'multi_head_number': 16, 'multi_head_output': 512, 'input_seq': 28, 'objective': 'mse', 'decoder_start_tok': 28}
best error
351.57428
mae:
529.8211


terapia_intensiva seq len 1 best config: 
{'layer_encoder': 4, 'layer_decoder': 2, 'multi_head_number': 8, 'multi_head_output': 512, 'input_seq': 28, 'objective': 'mse', 'decoder_start_tok': 56}
best error
1.5348816
mae:
13.610116
terapia_intensiva seq len 2 best config: 
{'layer_encoder': 4, 'layer_decoder': 2, 'multi_head_number': 16, 'multi_head_output': 512, 'input_seq': 28, 'objective': 'mse', 'decoder_start_tok': 56}
best error
0.5458908
mae:
5.1754956
terapia_intensiva seq len 7 best config: 
{'layer_encoder': 4, 'layer_decoder': 2, 'multi_head_number': 16, 'multi_head_output': 512, 'input_seq': 56, 'objective': 'mse', 'decoder_start_tok': 28}
best error
2.8885825
mae:
8.530022


terapia_intensiva seq len 14 best config: 
{'layer_encoder': 4, 'layer_decoder': 2, 'multi_head_number': 8, 'multi_head_output': 512, 'input_seq': 28, 'objective': 'mse', 'decoder_start_tok': 112}
best error
8.007038
mae:
15.386542


deceduti seq len 1 best config: 
{'layer_encoder': 2, 'layer_decoder': 2, 'multi_head_number': 16, 'multi_head_output': 512, 'input_seq': 56, 'objective': 'mse', 'decoder_start_tok': 28}
best error
0.008892059
mae:
2.2642758
deceduti seq len 2 best config: 
{'layer_encoder': 3, 'layer_decoder': 2, 'multi_head_number': 8, 'multi_head_output': 512, 'input_seq': 56, 'objective': 'mse', 'decoder_start_tok': 28}
best error
0.08192921
mae:
2.3583283
deceduti seq len 7 best config: 
{'layer_encoder': 3, 'layer_decoder': 2, 'multi_head_number': 8, 'multi_head_output': 512, 'input_seq': 28, 'objective': 'mse', 'decoder_start_tok': 28}
best error
1.2070245
mae:
2.7024374


deceduti seq len 14 best config: 
{'layer_encoder': 4, 'layer_decoder': 2, 'multi_head_number': 8, 'multi_head_output': 512, 'input_seq': 28, 'objective': 'mse', 'decoder_start_tok': 56}
best error
1.7455639
mae:
4.029673


##Risultati

###Deceduti
|    | LSTM                           | XGBoost                        | ARIMA                          | Informer                       | NeuralProphet                       |
|----|--------------------------------|--------------------------------|--------------------------------|--------------------------------|--------------------------------|
| 1  | Avg MAE: 3.18  <br />Best Slot: 0.03 | Avg MAE: 2.78  <br />Best Slot: 0.02 | Avg MAE: **2.42**  <br />Best Slot: 0.01 | Avg MAE: **2.26**  <br />Best Slot: 0.008| Avg MAE: 3.18 |
| 2  | Avg MAE: 4.03  <br />Best Slot: 0.33 | Avg MAE: 2.97  <br />Best Slot: 0.11 | Avg MAE: **2.45**  <br />Best Slot: 0.44 | Avg MAE: **2.35**  <br />Best Slot: 0.08 | Avg MAE: 6.94 |
| 7  | Avg MAE: **2.51**  <br />Best Slot: 1.01 | Avg MAE: 3.30  <br />Best Slot: 1.19 | Avg MAE: 2.53  <br />Best Slot: 1.06 | Avg MAE: 2.70  <br />Best Slot: 1.20 | Avg MAE: 6.73  |
| 14 | Avg MAE: 6.22  <br />Best Slot: 3.49 | Avg MAE: 3.31  <br />Best Slot: 1.62 | Avg MAE: **2.50**  <br />Best Slot: 1.21 | Avg MAE: 4.02  <br />Best Slot: 1.74 | Avg MAE: 2.75   |


###Terapia Intensiva
|    | LSTM                           | XGBoost                        | ARIMA                          | Informer                       | NeuralProphet                       |
|----|--------------------------------|--------------------------------|--------------------------------|--------------------------------|--------------------------------|
| 1  | Avg MAE: 4.36  <br />Best Slot: 0.03 | Avg MAE: 3.21  <br />Best Slot: 0.002 | Avg MAE: **2.31**  <br />Best Slot: 0.02 | Avg MAE: 13.61  <br />Best Slot: 1.53| Avg MAE: 16.22  |
| 2  | Avg MAE: 5.83  <br />Best Slot: 1.08 | Avg MAE: 4.55  <br />Best Slot: 0.47 | Avg MAE: **2.87**  <br />Best Slot: 0.05 | Avg MAE: 5.17  <br />Best Slot: 0.54 | Avg MAE: 3.28   |
| 7  | Avg MAE: 11.09  <br />Best Slot: 1.95 | Avg MAE: 7.03  <br />Best Slot: 1.89 | Avg MAE: 4.73  <br />Best Slot: 1.05 | Avg MAE: 8.53  <br />Best Slot: 2.88 | Avg MAE: **4.57**  |
| 14 | Avg MAE: 13.17  <br />Best Slot: 5.95 | Avg MAE: 9.37  <br />Best Slot: 2.96 | Avg MAE: **6.86**  <br />Best Slot: 1.22 | Avg MAE: 15.38  <br />Best Slot: 8.00| Avg MAE: 19.25   |

###Nuovi Positivi
|    | LSTM                              | XGBoost                           | ARIMA                             | Informer                       | NeuralProphet                       |
|----|-----------------------------------|-----------------------------------|-----------------------------------|--------------------------------|--------------------------------|
| 1  | Avg MAE: 118.94  <br />Best Slot: 0.03  | Avg MAE: 74.04  <br />Best Slot: 0.56   | Avg MAE: **57.19**  <br />Best Slot: 0.07   | Avg MAE: 118.74  <br />Best Slot: 1.82 | Avg MAE: 89.37  |
| 2  | Avg MAE: 113.78  <br />Best Slot: 6.82  | Avg MAE: 88.25  <br />Best Slot: 6.17   | Avg MAE: **65.80**  <br />Best Slot: 3.98   | Avg MAE: 90.73   <br />Best Slot: 4.62   | Avg MAE: 79.94  |
| 7  | Avg MAE: 132.39  <br />Best Slot: 33.74 | Avg MAE: 113.55  <br />Best Slot: 28.43 | Avg MAE: **85.05**  <br />Best Slot: 16.71  | Avg MAE: 162.37  <br />Best Slot: 41.94  | Avg MAE: 220.26  |
| 14 | Avg MAE: 156.96  <br />Best Slot: 48.71 | Avg MAE: 169.45  <br />Best Slot: 27.14 | Avg MAE: **114.03**  <br />Best Slot: 22.81 | Avg MAE: 529.82  <br />Best Slot: 351.47 | Avg MAE: 138.67   |

ARIMA mantiene un errore di previsione inferiore rispetto agli altri modelli in tutti gli scenari tranne per:
- Orizzonte deceduti di 7 giorni in cui LSTM ha MAE inferiore
- Orizzonte 1 e 2 giorni di deceduti in cui Informer ha un errore inferiore
- Terapia intensiva con orizzonte 7 giorni in cui l'errore inferiore è con NerualProphet.
